# Creación de un mapa de contaminación

Se pretende cargar la posición de las estaciones de medición de contaminantes junto con sus contaminantes y realizar una interpolación de estos contaminantes en todo un mallado entre las estaciones conocidas.

In [116]:
import numpy as np
import pandas as pd
import re
from scipy.interpolate import SmoothBivariateSpline
import matplotlib.pyplot as plt 
import pyarrow #uninstall libboost & conda install pyarrow=0.13.* -c conda-forge

In [117]:
def dms2dd(input):
    "Esta función tranforma un punto expresado en coordenadas geográficas a coordenadas UTM"
    degrees,minutes,seconds,direction=re.split('[º\´"]+',str(input))
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd;

Se leeran las coordenadas de las estaciones y se filtrarán las que son de nuestro interés

In [118]:
posicion_estaciones=pd.read_excel('./data/EstacionesLocalizacion.xlsx') #Se carga el fichero

Se filtra el fichero de estaciones a la estaciones de nuestro estudio

In [119]:
Estacion=[4,8,11,35,38,39,47,48,49,50]
posicion_estaciones=posicion_estaciones[posicion_estaciones['Estacion'].isin(Estacion)]
posicion_estaciones.set_index('Estacion',inplace=True)
posicion_estaciones

,Columna1,Latitud,Longitud
Estacion,,,
47,Mendez Alavaro,"40º23´53.17""N","3º41´12.57""O"
11,Av Ramon y Cajal,"40º27´5.29""N","3º40´38.5""O"
8,Escuelas Aguirre,"40º25´17.63""N","3º40´56.22""O"
49,Retiro,"40º24´52""N","3º40´57.3""O"
50,Plaza Castilla,"40º27´56.1""N","3º41´19.48""O"
48,Paseo Castellana,"40º26´23.61""N","3º41´25.34""O"
35,Plaza del carmen,"40º25´9.15""N","3º42´11.4""O"
38,Cuatro Caminos,"40º26´43.97""N","3º42´25.64""O"
39,Barrio del pilar,"40º28´41.64""N","3º42´41.53""O"


Se convertirán los valores de Latitud y Longitud en coordenadas decimales

In [120]:
posicion_estaciones[['Latitud','Longitud']]=posicion_estaciones[['Latitud','Longitud']].applymap(lambda x: dms2dd(x))

In [121]:
posicion_estaciones.sort_values(['Latitud','Longitud'])

,Columna1,Latitud,Longitud
Estacion,,,
47,Mendez Alavaro,40.398103,3.686825
49,Retiro,40.414444,3.682583
35,Plaza del carmen,40.419208,3.703167
8,Escuelas Aguirre,40.421564,3.682283
4,Plaza de España,40.423883,3.712197
48,Paseo Castellana,40.439892,3.690372
38,Cuatro Caminos,40.445547,3.707122
11,Av Ramon y Cajal,40.451469,3.677361
50,Plaza Castilla,40.465583,3.688744


In [122]:
sortLatitud=posicion_estaciones['Latitud'].sort_values()
sortLongitud=posicion_estaciones['Longitud'].sort_values()

Se desea obtener una matriz de puntos sobre la que realizar una interpolación. La interpolación se realizará sobre los puntos conocidos, para obtener una matriz de valores contaminantes, que representará la polución en cada espacio geografico de Madrid capital

In [123]:
Latitudes=[]
Longitudes=[]
for i in range(0,(len(sortLatitud))):
    if i==9:
        lastLat=np.round(sortLatitud.iloc[i],decimals=6)
        lastLon=np.round(sortLongitud.iloc[i],decimals=6)
        Latitudes.append(lastLat)
        Longitudes.append(lastLon)
    else:
        inicioLat=sortLatitud.iloc[i]
        inicioLon=sortLongitud.iloc[i]
        finLat=sortLatitud.iloc[i+1]
        finLon=sortLongitud.iloc[i+1]
        betweenLat=np.round(np.arange(inicioLat,finLat,0.001),decimals=6)
        betweenLon=np.round(np.arange(inicioLon,finLon,0.001),decimals=6)
        Latitudes.extend(betweenLat)
        Longitudes.extend(betweenLon)

In [124]:
datos_estaciones=pd.read_csv('./data/NO2_estaciones.csv')
datos_estaciones.head()

,FECHA,Estacion4,Estacion8,Estacion11,Estacion35,Estacion38,Estacion39,Estacion47,Estacion48,Estacion49,Estacion50
0,2015-01-01 01:00:00,83.0,168.0,93.0,109.0,102.0,104.0,112.0,108.0,136.0,127.0
1,2015-01-01 02:00:00,77.0,203.0,224.0,133.0,183.0,168.0,120.0,114.0,111.0,170.0
2,2015-01-01 03:00:00,86.0,220.0,152.0,126.0,193.0,203.0,109.0,141.0,109.0,118.0
3,2015-01-01 04:00:00,65.0,169.0,151.0,112.0,122.0,147.0,87.0,145.0,99.0,126.0
4,2015-01-01 05:00:00,53.0,154.0,106.0,109.0,97.0,159.0,76.0,142.0,103.0,140.0


In [125]:
fechas=list(datos_estaciones.FECHA)

In [126]:
fechas2015=[]
fechas2016=[]
fechas2017=[]
fechas2018=[]
fechas2019=[]
fechas2020=[]
for date in fechas:
    if re.findall('2015', date):
        fechas2015.append(date)
    elif re.findall('2016', date):
        fechas2016.append(date)
    elif re.findall('2017', date):
        fechas2017.append(date)
    elif re.findall('2018', date):
        fechas2018.append(date)
    elif re.findall('2019', date):
        fechas2019.append(date)
    else:
        fechas2020.append(date)

In [127]:
ix1 = pd.MultiIndex.from_product([fechas,Latitudes])

In [128]:
mapa = pd.DataFrame(index=ix1, columns=Longitudes, dtype=np.float32)

In [129]:
for ano in ['2016','2017','2018']:#,'2019','2020']:
    variable='fechas'+ano
    fechas=eval(variable)
    for date in fechas:
        print(date)
        x=[]
        y=[]
        Z=[]
        for i in Estacion:
            lat=posicion_estaciones.loc[i,'Latitud'].round(decimals=6)
            #print(lat)
            lon=posicion_estaciones.loc[i,'Longitud'].round(decimals=6)
            #print(lon)
            Est='Estacion'+str(i)
            sust=float(datos_estaciones[datos_estaciones['FECHA']==date][Est])
            #mapa.loc[(date,lat),lon]=sust
            x.append(lon)
            y.append(lat)
            Z.append(sust)
        #interp_spline=interpolate.interp2d(y,x,Z,kind='cubic')
        interp_spline = SmoothBivariateSpline(y,x,Z,kx=2,ky=2)
        mapa.loc[(date),:] = interp_spline(Latitudes,Longitudes)
    mapa_interp=mapa.stack().reset_index()
    mapa_interp.columns=['fecha','latitud','longitud','contaminacion']
    filename='./data/mapa_interpolado'+ano
    mapa_interp.to_feather(filename)

2016-01-01 00:00:00


/anaconda3/lib/python3.6/site-packages/scipy/interpolate/fitpack2.py:1057: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)


2016-01-01 01:00:00
2016-01-01 02:00:00
2016-01-01 03:00:00
2016-01-01 04:00:00
2016-01-01 05:00:00
2016-01-01 06:00:00
2016-01-01 07:00:00
2016-01-01 08:00:00
2016-01-01 09:00:00
2016-01-01 10:00:00
2016-01-01 11:00:00
2016-01-01 12:00:00
2016-01-01 13:00:00
2016-01-01 14:00:00
2016-01-01 15:00:00
2016-01-01 16:00:00
2016-01-01 17:00:00
2016-01-01 18:00:00
2016-01-01 19:00:00
2016-01-01 20:00:00
2016-01-01 21:00:00
2016-01-01 22:00:00
2016-01-01 23:00:00
2016-01-02 00:00:00
2016-01-02 01:00:00
2016-01-02 02:00:00
2016-01-02 03:00:00
2016-01-02 04:00:00
2016-01-02 05:00:00
2016-01-02 06:00:00
2016-01-02 07:00:00
2016-01-02 08:00:00
2016-01-02 09:00:00
2016-01-02 10:00:00
2016-01-02 11:00:00
2016-01-02 12:00:00
2016-01-02 13:00:00
2016-01-02 14:00:00
2016-01-02 15:00:00
2016-01-02 16:00:00
2016-01-02 17:00:00
2016-01-02 18:00:00
2016-01-02 19:00:00
2016-01-02 20:00:00
2016-01-02 21:00:00
2016-01-02 22:00:00
2016-01-02 23:00:00
2016-01-03 00:00:00
2016-01-03 01:00:00
2016-01-03 02:00:00


2016-01-18 04:00:00
2016-01-18 05:00:00
2016-01-18 06:00:00
2016-01-18 07:00:00
2016-01-18 08:00:00
2016-01-18 09:00:00
2016-01-18 10:00:00
2016-01-18 11:00:00
2016-01-18 12:00:00
2016-01-18 13:00:00
2016-01-18 14:00:00
2016-01-18 15:00:00
2016-01-18 16:00:00
2016-01-18 17:00:00
2016-01-18 18:00:00
2016-01-18 19:00:00
2016-01-18 20:00:00
2016-01-18 21:00:00
2016-01-18 22:00:00
2016-01-18 23:00:00
2016-01-19 00:00:00
2016-01-19 01:00:00
2016-01-19 02:00:00
2016-01-19 03:00:00
2016-01-19 04:00:00
2016-01-19 05:00:00
2016-01-19 06:00:00
2016-01-19 07:00:00
2016-01-19 08:00:00
2016-01-19 09:00:00
2016-01-19 10:00:00
2016-01-19 11:00:00
2016-01-19 12:00:00
2016-01-19 13:00:00
2016-01-19 14:00:00
2016-01-19 15:00:00
2016-01-19 16:00:00
2016-01-19 17:00:00
2016-01-19 18:00:00
2016-01-19 19:00:00
2016-01-19 20:00:00
2016-01-19 21:00:00
2016-01-19 22:00:00
2016-01-19 23:00:00
2016-01-20 00:00:00
2016-01-20 01:00:00
2016-01-20 02:00:00
2016-01-20 03:00:00
2016-01-20 04:00:00
2016-01-20 05:00:00


2016-02-04 07:00:00
2016-02-04 08:00:00
2016-02-04 09:00:00
2016-02-04 10:00:00
2016-02-04 11:00:00
2016-02-04 12:00:00
2016-02-04 13:00:00
2016-02-04 14:00:00
2016-02-04 15:00:00
2016-02-04 16:00:00
2016-02-04 17:00:00
2016-02-04 18:00:00
2016-02-04 19:00:00
2016-02-04 20:00:00
2016-02-04 21:00:00
2016-02-04 22:00:00
2016-02-04 23:00:00
2016-02-05 00:00:00
2016-02-05 01:00:00
2016-02-05 02:00:00
2016-02-05 03:00:00
2016-02-05 04:00:00
2016-02-05 05:00:00
2016-02-05 06:00:00
2016-02-05 07:00:00
2016-02-05 08:00:00
2016-02-05 09:00:00
2016-02-05 10:00:00
2016-02-05 11:00:00
2016-02-05 12:00:00
2016-02-05 13:00:00
2016-02-05 14:00:00
2016-02-05 15:00:00
2016-02-05 16:00:00
2016-02-05 17:00:00
2016-02-05 18:00:00
2016-02-05 19:00:00
2016-02-05 20:00:00
2016-02-05 21:00:00
2016-02-05 22:00:00
2016-02-05 23:00:00
2016-02-06 00:00:00
2016-02-06 01:00:00
2016-02-06 02:00:00
2016-02-06 03:00:00
2016-02-06 04:00:00
2016-02-06 05:00:00
2016-02-06 06:00:00
2016-02-06 07:00:00
2016-02-06 08:00:00


...

2018-10-23 01:00:00
2018-10-23 02:00:00
2018-10-23 03:00:00
2018-10-23 04:00:00
2018-10-23 05:00:00
2018-10-23 06:00:00
2018-10-23 07:00:00
2018-10-23 08:00:00
2018-10-23 09:00:00
2018-10-23 10:00:00
2018-10-23 11:00:00
2018-10-23 12:00:00
2018-10-23 13:00:00
2018-10-23 14:00:00
2018-10-23 15:00:00
2018-10-23 16:00:00
2018-10-23 17:00:00
2018-10-23 18:00:00
2018-10-23 19:00:00
2018-10-23 20:00:00
2018-10-23 21:00:00
2018-10-23 22:00:00
2018-10-23 23:00:00
2018-10-24 00:00:00
2018-10-24 01:00:00
2018-10-24 02:00:00
2018-10-24 03:00:00
2018-10-24 04:00:00
2018-10-24 05:00:00
2018-10-24 06:00:00
2018-10-24 07:00:00
2018-10-24 08:00:00
2018-10-24 09:00:00
2018-10-24 10:00:00
2018-10-24 11:00:00
2018-10-24 12:00:00
2018-10-24 13:00:00
2018-10-24 14:00:00
2018-10-24 15:00:00
2018-10-24 16:00:00
2018-10-24 17:00:00
2018-10-24 18:00:00
2018-10-24 19:00:00
2018-10-24 20:00:00
2018-10-24 21:00:00
2018-10-24 22:00:00
2018-10-24 23:00:00
2018-10-25 00:00:00
2018-10-25 01:00:00
2018-10-25 02:00:00


2018-11-09 03:00:00
2018-11-09 04:00:00
2018-11-09 05:00:00
2018-11-09 06:00:00
2018-11-09 07:00:00
2018-11-09 08:00:00
2018-11-09 09:00:00
2018-11-09 10:00:00
2018-11-09 11:00:00
2018-11-09 12:00:00
2018-11-09 13:00:00
2018-11-09 14:00:00
2018-11-09 15:00:00
2018-11-09 16:00:00
2018-11-09 17:00:00
2018-11-09 18:00:00
2018-11-09 19:00:00
2018-11-09 20:00:00
2018-11-09 21:00:00
2018-11-09 22:00:00
2018-11-09 23:00:00
2018-11-10 00:00:00
2018-11-10 01:00:00
2018-11-10 02:00:00
2018-11-10 03:00:00
2018-11-10 04:00:00
2018-11-10 05:00:00
2018-11-10 06:00:00
2018-11-10 07:00:00
2018-11-10 08:00:00
2018-11-10 09:00:00
2018-11-10 10:00:00
2018-11-10 11:00:00
2018-11-10 12:00:00
2018-11-10 13:00:00
2018-11-10 14:00:00
2018-11-10 15:00:00
2018-11-10 16:00:00
2018-11-10 17:00:00
2018-11-10 18:00:00
2018-11-10 19:00:00
2018-11-10 20:00:00
2018-11-10 21:00:00
2018-11-10 22:00:00
2018-11-10 23:00:00
2018-11-11 00:00:00
2018-11-11 01:00:00
2018-11-11 02:00:00
2018-11-11 03:00:00
2018-11-11 04:00:00


2018-11-26 05:00:00
2018-11-26 06:00:00
2018-11-26 07:00:00
2018-11-26 08:00:00
2018-11-26 09:00:00
2018-11-26 10:00:00
2018-11-26 11:00:00
2018-11-26 12:00:00
2018-11-26 13:00:00
2018-11-26 14:00:00
2018-11-26 15:00:00
2018-11-26 16:00:00
2018-11-26 17:00:00
2018-11-26 18:00:00
2018-11-26 19:00:00
2018-11-26 20:00:00
2018-11-26 21:00:00
2018-11-26 22:00:00
2018-11-26 23:00:00
2018-11-27 00:00:00
2018-11-27 01:00:00
2018-11-27 02:00:00
2018-11-27 03:00:00
2018-11-27 04:00:00
2018-11-27 05:00:00
2018-11-27 06:00:00
2018-11-27 07:00:00
2018-11-27 08:00:00
2018-11-27 09:00:00
2018-11-27 10:00:00
2018-11-27 11:00:00
2018-11-27 12:00:00
2018-11-27 13:00:00
2018-11-27 14:00:00
2018-11-27 15:00:00
2018-11-27 16:00:00
2018-11-27 17:00:00
2018-11-27 18:00:00
2018-11-27 19:00:00
2018-11-27 20:00:00
2018-11-27 21:00:00
2018-11-27 22:00:00
2018-11-27 23:00:00
2018-11-28 00:00:00
2018-11-28 01:00:00
2018-11-28 02:00:00
2018-11-28 03:00:00
2018-11-28 04:00:00
2018-11-28 05:00:00
2018-11-28 06:00:00


2018-12-13 07:00:00
2018-12-13 08:00:00
2018-12-13 09:00:00
2018-12-13 10:00:00
2018-12-13 11:00:00
2018-12-13 12:00:00
2018-12-13 13:00:00
2018-12-13 14:00:00
2018-12-13 15:00:00
2018-12-13 16:00:00
2018-12-13 17:00:00
2018-12-13 18:00:00
2018-12-13 19:00:00
2018-12-13 20:00:00
2018-12-13 21:00:00
2018-12-13 22:00:00
2018-12-13 23:00:00
2018-12-14 00:00:00
2018-12-14 01:00:00
2018-12-14 02:00:00
2018-12-14 03:00:00
2018-12-14 04:00:00
2018-12-14 05:00:00
2018-12-14 06:00:00
2018-12-14 07:00:00
2018-12-14 08:00:00
2018-12-14 09:00:00
2018-12-14 10:00:00
2018-12-14 11:00:00
2018-12-14 12:00:00
2018-12-14 13:00:00
2018-12-14 14:00:00
2018-12-14 15:00:00
2018-12-14 16:00:00
2018-12-14 17:00:00
2018-12-14 18:00:00
2018-12-14 19:00:00
2018-12-14 20:00:00
2018-12-14 21:00:00
2018-12-14 22:00:00
2018-12-14 23:00:00
2018-12-15 00:00:00
2018-12-15 01:00:00
2018-12-15 02:00:00
2018-12-15 03:00:00
2018-12-15 04:00:00
2018-12-15 05:00:00
2018-12-15 06:00:00
2018-12-15 07:00:00
2018-12-15 08:00:00


2018-12-30 09:00:00
2018-12-30 10:00:00
2018-12-30 11:00:00
2018-12-30 12:00:00
2018-12-30 13:00:00
2018-12-30 14:00:00
2018-12-30 15:00:00
2018-12-30 16:00:00
2018-12-30 17:00:00
2018-12-30 18:00:00
2018-12-30 19:00:00
2018-12-30 20:00:00
2018-12-30 21:00:00
2018-12-30 22:00:00
2018-12-30 23:00:00
2018-12-31 00:00:00
2018-12-31 01:00:00
2018-12-31 02:00:00
2018-12-31 03:00:00
2018-12-31 04:00:00
2018-12-31 05:00:00
2018-12-31 06:00:00
2018-12-31 07:00:00
2018-12-31 08:00:00
2018-12-31 09:00:00
2018-12-31 10:00:00
2018-12-31 11:00:00
2018-12-31 12:00:00
2018-12-31 13:00:00
2018-12-31 14:00:00
2018-12-31 15:00:00
2018-12-31 16:00:00
2018-12-31 17:00:00
2018-12-31 18:00:00
2018-12-31 19:00:00
2018-12-31 20:00:00
2018-12-31 21:00:00
2018-12-31 22:00:00
2018-12-31 23:00:00


In [105]:
mapa_interp

,fecha,longitud,latitud,contaminacion
0,2016-01-01 00:00:00,40.398103,3.677361,89.803139
1,2016-01-01 00:00:00,40.398103,3.678361,85.614761
2,2016-01-01 00:00:00,40.398103,3.679361,81.465973
3,2016-01-01 00:00:00,40.398103,3.680361,77.356766
4,2016-01-01 00:00:00,40.398103,3.681361,73.287148
5,2016-01-01 00:00:00,40.398103,3.682283,69.570030
6,2016-01-01 00:00:00,40.398103,3.682583,68.367813
7,2016-01-01 00:00:00,40.398103,3.683583,64.386154
8,2016-01-01 00:00:00,40.398103,3.684583,60.444077
9,2016-01-01 00:00:00,40.398103,3.685583,56.541588


In [156]:
mapa2016=pd.read_feather('./data/mapa_interpolado2016')

In [168]:
%%time
mapa2016=pd.read_feather('./data/mapa_interpolado2016')
mapa2016.set_index(['fecha','longitud','latitud'],inplace=True)
mapa2016.sort_index(inplace=True)
mapa2016=mapa2016.unstack('longitud')

CPU times: user 1min 17s, sys: 14.9 s, total: 1min 31s
Wall time: 1min 31s


In [170]:
#se hace la media de los valores por mes
mes=mapa2016.query("fecha >= '2016-01-01 00:00:00' & fecha > '2016-02-01 00:00:00'").mean(level='latitud')
mes

contaminacion                                                  \
longitud       3.677361    3.678361    3.679361    3.680361    3.681361   
latitud                                                                   
40.398103   -235.786057 -199.470078 -164.914795 -132.120224 -101.086342   
40.399103   -217.392487 -183.370514 -150.999985 -120.280922  -91.213326   
40.400103   -199.501190 -167.712051 -137.467972 -108.768997  -81.615105   
40.401103   -182.112183 -152.494690 -124.318771  -97.584442  -72.291695   
40.402103   -165.225449 -137.718430 -111.552368  -86.727257  -63.243092   
40.403103   -148.840988 -123.383286  -99.168770  -76.197441  -54.469292   
40.404103   -132.958801 -109.489235  -87.167969  -65.994987  -45.970299   
40.405103   -117.578888  -96.036301  -75.549973  -56.119907  -37.746109   
40.406103   -102.701263  -83.024460  -64.314774  -46.572197  -29.796726   
40.407103    -88.325905  -70.453735  -53.462379  -37.351852  -22.122150   
40.408103    -74.452835  -58.324104  -42.992790  -28.458879  -14.722380   
40.409103    -61.082031  -46.635586  -32.905998  -19.893274   -7.597414   
40.410103    -48.213512  -35.388168  -23.202011  -11.655040   -0.747254   
40.411103    -35.847263  -24.581856  -13.880824   -3.744173    5.828099   
40.412103    -23.983295  -14.216647   -4.942440    3.839325   12.128648   
40.413103    -12.621605   -4.292544    3.613142   11.095453   18.154390   
40.414103     -1.762190    5.190455   11.785922   18.024214   23.905327   
40.414444      1.826030    8.323303   14.485316   20.312071   25.803564   
40.415444     12.011891   17.214781   22.144760   26.801828   31.185986   
40.416444     21.695477   25.665154   29.421400   32.964218   36.293602   
40.417444     30.876783   33.674423   36.315239   38.799236   41.126415   
40.418444     39.555817   41.242584   42.826275   44.306889   45.684418   
40.419208     45.848137   46.727425   47.542759   48.294136   48.981556   
40.420208     53.641151   53.517479   53.378532   53.224308   53.054806   
40.421208     60.931889   59.866432   58.831505   57.827110   56.853249   
40.421564     63.406158   62.020187   60.680367   59.386692   58.139164   
40.422564     70.015808   67.771004   65.614258   63.545582   61.564968   
40.423564     76.123184   73.080711   70.165352   67.377106   64.715965   
40.423883     77.965767   74.681709   71.536613   68.530487   65.663322   
40.424883     83.410637   79.409599   75.582787   71.930206   68.451859   
...                 ...         ...         ...         ...         ...   
40.450547     51.292519   49.821190   48.447208   47.170567   45.991268   
40.451469     43.982670   43.351963   42.780663   42.268776   41.816296   
40.452469     35.571728   35.911541   36.266865   36.637695   37.024036   
40.453469     26.658512   28.030018   29.370264   30.679249   31.956972   
40.454469     17.243019   19.707390   22.090860   24.393431   26.615101   
40.455469      7.325248   10.943656   14.428655   17.780245   20.998425   
40.456469     -3.094799    1.738818    6.383648   10.839689   15.106943   
40.457469    -14.017123   -7.907124   -2.044161    3.571765    8.940655   
40.458469    -25.441725  -17.994171  -10.854773   -4.023528    2.499562   
40.459469    -37.368603  -28.522324  -20.048185  -11.946190   -4.216338   
40.460469    -49.797756  -39.491577  -29.624401  -20.196222  -11.207043   
40.461469    -62.729191  -50.901939  -39.583416  -28.773623  -18.472553   
40.462469    -76.162895  -62.753407  -49.925236  -37.678391  -26.012869   
40.463469    -90.098885  -75.045975  -60.649857  -46.910530  -33.827991   
40.464469   -104.537148  -87.779648  -71.757278  -56.470036  -41.917919   
40.465469   -119.477684 -100.954430  -83.247505  -66.356911  -50.282654   
40.465583   -121.212799 -102.484360  -84.581696  -67.504807  -51.253681   
40.466583   -136.712875 -116.150536  -96.498360  -77.756371  -59.924549   
40.467583   -152.715225 -130.257812 -108.797829  -88.335304  -68.870224   
40.468583   -169.219864 -144.806183

Para poder pasar los datos generados a una versión ordenada, que según Hadley Wickham, deberá realizar un "melt"

In [ ]:
mapa_interp=mapa.stack().reset_index()